## Midterm
### Due Friday, October 27 by midnight

This midterm uses the 'nyc_health.db'.

Questions are worth 10 points each. You'll get an additional 10 points just for making the submission!

Tips:
- Work slow; tackle the queries one small piece at a time
- Remember to include the code to create any views you use
- Make sure your tables and their column names are the same as we used in class (i.e., your CREATE TABLE statements should be identical to what we wrote in class). You can name calculated fields whatever you want.
- Double and triple check your .data302 file before submitting

In [225]:
import sqlite3
import pandas as pd
import dbga
from collections import defaultdict as dd
import json

dbga.__version__ 

'1.0.3'

In [226]:
conn = sqlite3.connect('nyc_health.db')
curs = conn.cursor()

In [227]:
my_sql = dd(dict)

In [228]:
pd.read_sql("PRAGMA table_info(tRest);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,camis,INTEGER,0,None,1
1,1,dba,TEXT,1,None,0
2,2,boro,TEXT,1,None,0
3,3,building,TEXT,1,None,0
4,4,street,TEXT,1,None,0
5,5,zip,TEXT,1,None,0
6,6,phone,TEXT,1,None,0
7,7,cuisine,TEXT,1,None,0


In [229]:
pd.read_sql("PRAGMA table_info(tInsp);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,camis,INTEGER,1,None,1
1,1,viol_id,TEXT,1,None,2
2,2,action_id,INTEGER,1,None,3
3,3,date,TEXT,1,None,4
4,4,time,TEXT,1,None,5


In [230]:
pd.read_sql("PRAGMA table_info(tViol);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,viol_id,TEXT,0,None,1
1,1,viol_desc,TEXT,1,None,0


In [231]:
pd.read_sql("PRAGMA table_info(tAction);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,action_id,INTEGER,0,None,1
1,1,action_desc,TEXT,1,None,0


In [232]:
pd.read_sql("SELECT * FROM sqlite_master;", conn)

,type,name,tbl_name,rootpage,sql
0,table,tRest,tRest,2,CREATE TABLE tRest (\n camis INTEGER PRIMAR...
1,table,tAction,tAction,3,CREATE TABLE tAction (\n action_id INTEGER ...
2,table,tViol,tViol,4,CREATE TABLE tViol (\n viol_id TEXT PRIMARY...
3,index,sqlite_autoindex_tViol_1,tViol,5,None
4,table,tInsp,tInsp,6,CREATE TABLE tInsp (\n camis INTEGER NOT NU...
5,index,sqlite_autoindex_tInsp_1,tInsp,7,None


---
1) Get a list of all restaurants (by camis) who have been cited for a violation containing the words 'vermin', 'mice', or 'rats'.

Have your query return the following columns:
- camis
- dba
- viol_id
- viol_desc
- A count of the number of times they have been cited for each violation

Order the results with the most violations on top.

In [233]:
sql1 = """ 
SELECT camis, dba, viol_id, viol_desc, count(*) as NumViol
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
WHERE viol_desc LIKE '%vermin%' OR viol_desc LIKE '%mice%' OR viol_desc LIKE '%rats%'
GROUP BY camis
ORDER BY NumViol DESC
;"""
pd.read_sql(sql1, conn)

,camis,dba,viol_id,viol_desc,NumViol
0,40861669,NEW VICTORY RESTAURANT,04K,Evidence of rats or live rats present in facil...,32
1,50033122,PARTY WELL REST & ORIENTAL BAKERY,08A,Facility not vermin proof. Harborage or condit...,26
2,41691396,SAM'S,04L,Evidence of mice or live mice present in facil...,24
3,50004065,RINCONCITO FAMILIAR MEXICAN CUISINE,08A,Facility not vermin proof. Harborage or condit...,23
4,41642733,TACO KING,04L,Evidence of mice or live mice present in facil...,23
...,...,...,...,...,...
16186,40364404,POLISH NATIONAL HOME,08A,Facility not vermin proof. Harborage or condit...,1
16187,40363427,BAGELS N BUNS,08A,Facility not vermin proof. Harborage or condit...,1
16188,40363298,CAFE METRO,08A,Facility not vermin proof. Harborage or condit...,1
16189,40359705,NATHAN'S FAMOUS,08A,Facility not vermin proof. Harborage or condit...,1


In [234]:
my_sql['q1']['sql'] = sql1

---
2) For each different cuisine type, determine the most commonly cited violation **excluding viol_id 10F** (this is by far the most common violation, and not very "gross").

Have your query return:

- cuisine
- viol_id
- viol_desc
- Number of violations for their most common violation (except 10F)

Limit those results to cuisine types where the max number of violations is greater than 1000.

Order the results in ascending order by the cuisine type.

In [235]:
sql2 = """ 
SELECT cuisine, viol_id, viol_desc, count(*) as NumViol
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
WHERE viol_id != '10F'
GROUP BY cuisine
HAVING NumViol > 1000
ORDER BY cuisine 
;"""
pd.read_sql(sql2, conn)

,cuisine,viol_id,viol_desc,NumViol
0,African,02B,Hot food item not held at or above 140Âº F.,1260
1,American,04M,Live roaches present in facility's food and/or...,76289
2,Asian,10B,Plumbing not properly installed or maintained;...,5414
3,Bagels/Pretzels,06D,"Food contact surface not properly washed, rins...",2317
4,Bakery,22C,"Bulb not shielded or shatterproof, in areas wh...",10064
5,CafÃ©/Coffee/Tea,02G,Cold food item held above 41Âº F (smoked fish ...,12410
6,Caribbean,04N,Filth flies or food/refuse/sewage-associated (...,11219
7,Chicken,02G,Cold food item held above 41Âº F (smoked fish ...,5016
8,Chinese,02G,Cold food item held above 41Âº F (smoked fish ...,35968
9,Delicatessen,02B,Hot food item not held at or above 140Âº F.,5302


In [236]:
my_sql['q2']['sql'] = sql2

---

3) When the action description contains the word 'closed', what are the top 5 associated violations?


Have your query return:

- action_desc
- viol_desc
- total number of violations

Order the results with the largest number of violations on top

In [237]:
sql3 = """ 
SELECT action_desc, viol_desc, count(*) as NumViol
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE action_desc LIKE '%closed%'
GROUP BY camis
ORDER BY NumViol DESC
LIMIT 5
;"""
pd.read_sql(sql3, conn)

,action_desc,viol_desc,NumViol
0,Establishment Closed by DOHMH. Violations wer...,Hot food item not held at or above 140Âº F.,32
1,Establishment Closed by DOHMH. Violations wer...,Cold food item held above 41Âº F (smoked fish ...,25
2,Establishment Closed by DOHMH. Violations wer...,Hot food item not held at or above 140Âº F.,22
3,Establishment Closed by DOHMH. Violations wer...,Hot food item not held at or above 140Âº F.,22
4,Establishment Closed by DOHMH. Violations wer...,Hot food item not held at or above 140Âº F.,22


In [238]:
my_sql['q3']['sql'] = sql3

---

4) Determine which restaurant has been closed the most (action description contains 'closed').

For that camis, have your query return:

- camis
- dba
- viol_desc
- total number of violations per violation type

Include all violations for that camis (not just the ones they were closed for)

Order the results with the most common violation on top

In [239]:
sql4 = """ 
SELECT camis, dba, viol_desc, count(*) as NumViol
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE action_desc LIKE '%closed%'
GROUP BY camis
ORDER BY NumViol DESC
LIMIT 1
;"""
pd.read_sql(sql4, conn)

,camis,dba,viol_desc,NumViol
0,40922818,EL TAPATIO MEXICAN RESTAURANT,Facility not vermin proof. Harborage or condit...,32


In [240]:
my_sql['q4']['sql'] = sql4

---

5) Group all restaurants by their DBA, and computer the average number of violations per location.

For example, if there were 100 restaurants with the dba "STARBUCKS", and they had a total of 123 violations, then the average number of violations per location would be 123/100=1.23.

Have your query return:
- dba
- Number of locations
- Number of violations
- Avg violations per location

Order the results with the largest number of locations on top

_Watch out for integer division!_

In [241]:
sql5 = """ 
WITH
LocAndViol as
(
    SELECT dba, count(DISTINCT camis) as NumLoc, count(*) as NumViol
    FROM tRest
    JOIN tInsp USING(camis)
    JOIN tViol USING(viol_id)
    JOIN tAction USING(action_id)
    GROUP BY dba
)

SELECT *, 1.*NumViol/NumLoc as ViolPerLoc
FROM LocAndViol
ORDER BY NumLoc DESC
;"""
pd.read_sql(sql5, conn)

,dba,NumLoc,NumViol,ViolPerLoc
0,DUNKIN' DONUTS,410,4247,10.358537
1,SUBWAY,352,3729,10.593750
2,STARBUCKS,281,2130,7.580071
3,MCDONALD'S,206,2778,13.485437
4,"DUNKIN' DONUTS, BASKIN ROBBINS",110,1462,13.290909
...,...,...,...,...
20242,cloud social,1,11,11.000000
20243,energy fuel,1,8,8.000000
20244,re.union,1,14,14.000000
20245,taci's beyti restaurant,1,9,9.000000


In [242]:
my_sql['q5']['sql'] = sql5

---

6) For restaurants (by DBA) with only a single location (i.e., DBA with exactly 1 associated CAMIS) and more than 10 total violations, what is their top violation, and how many times have they been cited for it?

Note that a restaurant might have been cited 1 time each for 10 different types of violations, and 2 times for another violation. That would be 12 total violations, which counts as more than 10.

**Limit your results** to cases where their top violation occurred greater than or equal to 15 times.

Have your query return:

- camis
- dba
- viol_desc
- number of violations

Order your results by number of violations, descending.

In [243]:
sql6 = """
WITH
ViolCounts  as
(
    SELECT camis, viol_desc, count(*) AS NumViol
    FROM tRest
    JOIN tInsp USING(camis)
    JOIN tViol USING(viol_id)
    GROUP BY camis
    HAVING NumViol >= 15
),

RestStats as
(
    SELECT camis, dba, sum(NumViol) AS TotalViolations
    FROM tRest
    JOIN ViolCounts USING(camis)
    GROUP BY camis, dba
    HAVING TotalViolations > 10
)

SELECT camis, dba, viol_desc, NumViol
FROM RestStats
JOIN ViolCounts USING(camis)
ORDER BY NumViol DESC
;"""
pd.read_sql(sql6, conn)

,camis,dba,viol_desc,NumViol
0,41683816,MAX BAKERY & RESTAURANT,Plumbing not properly installed or maintained;...,95
1,50001880,GANG SAN DEUL,"Food contact surface not properly washed, rins...",95
2,40965177,LA GATA GOLOSA #2,Food not protected from potential source of co...,91
3,50033122,PARTY WELL REST & ORIENTAL BAKERY,Plumbing not properly installed or maintained;...,91
4,41459659,T. K. KITCHEN,Cold food item held above 41Âº F (smoked fish ...,90
...,...,...,...,...
11171,50058911,CAFFEBENE,Plumbing not properly installed or maintained;...,15
11172,50063585,GARDEN DELI COFFEE,"No facilities available to wash, rinse and san...",15
11173,50063996,MCDONALD'S,Cold food item held above 41Âº F (smoked fish ...,15
11174,50066465,BARI'S PIZZA,Facility not vermin proof. Harborage or condit...,15


In [244]:
my_sql['q6']['sql'] = sql6

---

7) Considering all possible combinations of violation/action, how many of those combinations **do not occur** in the data?

Have your query return just the total number.

In [245]:
sql7 = """
WITH
PossibleCombos as (
    SELECT DISTINCT viol_id, action_id
    FROM tViol
    CROSS JOIN tAction
)
SELECT count(*) as TotalMissingCombos
FROM PossibleCombos
LEFT JOIN tInsp USING(viol_id)
WHERE viol_id IS NULL
;"""
pd.read_sql(sql7, conn)

,TotalMissingCombos
0,0


In [246]:
my_sql['q7']['sql'] = sql7

---

8) Return the action and violation descriptions associated with the previous question.

Have your query return:

- action_desc
- viol_desc

Order your results in ascending order by viol_desc

In [247]:
sql8 = """
WITH
PossibleCombos as (
    SELECT DISTINCT viol_id, action_id
    FROM tViol
    CROSS JOIN tAction
),

Descriptions as (
    SELECT viol_id, action_id, viol_desc, action_desc
    FROM PossibleCombos
    LEFT JOIN tViol USING(viol_id)
    LEFT JOIN tAction USING(action_id)
    WHERE viol_id IS NULL OR action_id IS NULL    
)
SELECT action_desc, viol_desc
FROM Descriptions
ORDER BY viol_desc ASC
;"""
pd.read_sql(sql8, conn)

,action_desc,viol_desc


In [248]:
my_sql['q8']['sql'] = sql8

---

9) For all restaurants (by camis) that have been cited for 90 or more violations, what is their top violation?

Have your query return:
- camis
- dba
- viol_id
- viol_desc
- Number of violations (for the top violation only)

Order the results in ascending order by dba


In [249]:
sql9 = """
WITH
ViolCounts as (
    SELECT camis, viol_id, viol_desc, COUNT(*) AS NumViol
    FROM tRest 
    JOIN tInsp USING(camis)
    JOIN tViol USING(viol_id)
    GROUP BY camis
    HAVING NumViol >= 90    
)
SELECT camis, dba, viol_id, viol_desc, MAX(NumViol) AS NumViol
FROM tRest 
JOIN ViolCounts USING(camis)
GROUP BY camis
ORDER BY dba ASC
;"""
pd.read_sql(sql9, conn)

,camis,dba,viol_id,viol_desc,NumViol
0,50001880,GANG SAN DEUL,06D,"Food contact surface not properly washed, rins...",95
1,40965177,LA GATA GOLOSA #2,06C,Food not protected from potential source of co...,91
2,41683816,MAX BAKERY & RESTAURANT,10B,Plumbing not properly installed or maintained;...,95
3,50033122,PARTY WELL REST & ORIENTAL BAKERY,10B,Plumbing not properly installed or maintained;...,91
4,41459659,T. K. KITCHEN,02G,Cold food item held above 41Âº F (smoked fish ...,90


In [250]:
my_sql['q9']['sql'] = sql9

In [251]:
with open('Midterm.data302', 'w') as f:
    json.dump(my_sql, f)

In [252]:
dbga.validate_submission_format(path_db = '../databases/nyc_health.db',
                                path_submission = 'Midterm.data302',
                                num_questions = 9)

Report saved to:  VALIDATION_Midterm.txt
